In [ ]:
from src.app import App
from src.obj.omdb_response import get_by_title

In [2]:
app = App()

In [3]:
ents = App.load_entries()

## Comparing my mean rating with the IMDB rating

In [ ]:
import plotly.graph_objects as go
from tqdm import tqdm

In [ ]:
db_results = {}

for group in tqdm(app.get_groups()):
    db_res = get_by_title(group.title)
    db_results[group.title] = (group, db_res)

In [ ]:
my_ratings = []
imdb_ratings = []
titles = []


for title, res in db_results.items():
    group, db_res = res
    if db_res is None:
        continue
    if db_res.imdb_rating is None:
        print(f"IMDB rating is None for {title}")
        continue
    imdb_ratings.append(db_res.imdb_rating)
    my_ratings.append(group.mean_rating)
    titles.append(title)

In [41]:
def best_fit(x_coors, y_coors):
    assert (n := len(x_coors)) == len(y_coors)
    xbar = sum(x_coors) / len(x_coors)
    ybar = sum(y_coors) / len(y_coors)

    numer = sum([xi * yi for xi, yi in zip(x_coors, y_coors)]) - n * xbar * ybar
    denum = sum([xi**2 for xi in x_coors]) - n * xbar**2

    b = numer / denum
    a = ybar - b * xbar

    return a, b

In [42]:
a, b = best_fit(imdb_ratings, my_ratings)
x_vals = [min(imdb_ratings) - 1, max(imdb_ratings) + 1]
y_vals = [a + b * x for x in x_vals]

In [44]:
fig = go.Figure(
    data=go.Scatter(y=my_ratings, x=imdb_ratings, mode="markers", text=titles)
)
fig.add_trace(
    go.Scatter(
        x=x_vals,
        y=y_vals,
        mode="lines",
        name=f"best fit line: y = {a:.2f} + {b:.2f}x",
    )
)
fig.update_layout(
    title="my ratings vs IMDB ratings",
    xaxis_title="IMDB rating",
    yaxis_title="My rating",
    margin=dict(l=10, r=10, t=30, b=10),
    template="plotly_dark",
    legend=dict(x=0, y=1, traceorder="normal", orientation="v"),
)
fig.show()

In [5]:
from src.obj.images_manager import ImagesStore, IMAGES_SERIES_BUCKET_NAME, FOLDER_NAME

In [18]:
store = ImagesStore([])

In [23]:
response = store._s3.list_objects_v2(
            Bucket=IMAGES_SERIES_BUCKET_NAME,
            Prefix=FOLDER_NAME + "/",
        )


In [24]:
images = response.get("Contents", [])

In [27]:
seen_etags = set()

for image in images:
    if image.get("ETag") not in seen_etags:
        print(f"{image.get('Size')}, {image.get('ETag').strip('"')!r}, {image.get('Key')}")
        seen_etags.add(image.get("ETag"))
    else:
        print("Duplicate image found:", image.get("Key"))

343892, 'a4bd608919b2c3af4b34fb84bd864660', movies-series-images/2025-08-22T18:12:23.360630+00:00.png
282963, '2807f6dd570ef20ee1b67c0405e4e6c7', movies-series-images/2025-08-22T18:53:39.679834+00:00.png
313570, '2578e573c8d611e93809e580e2d0ced1', movies-series-images/2025-08-22T18:57:01.905681+00:00.png
319183, 'fb36af9cd99ee83bf9bb0a8459b05b00', movies-series-images/2025-08-22T18:59:39.385849+00:00.png
340470, '3b575dfffd0dada94bc19d6b8e90922e', movies-series-images/2025-08-22T19:00:32.033455+00:00.png
697048, '30419672f35d13781577ed6ae648d15b', movies-series-images/2025-08-22T19:33:00.072318+00:00.png
376597, '6d118e8b46c1b17d435cda8726617b1b', movies-series-images/2025-08-22T19:35:53.564845+00:00.png
608664, '4c54b056aaa83c752ec70ec52e40d1c4', movies-series-images/2025-08-22T19:36:52.176472+00:00.png
1953469, '92d5466fbe0c4340040afb1b94f10861', movies-series-images/2025-08-22T20:26:26.904004+00:00.png
611172, '64c283b9fab665524a1b1ced0d508877', movies-series-images/2025-08-22T20:33